# Метрики графа: авторы, публикации, формулы

Лёгкая аналитика по **Person**, **Publication** и **Formula**: топы по связям со статьями, средние показатели, распределение по тезаурусам (FME / ME).

## 1. Загрузка данных

In [1]:
import sys
from pathlib import Path
from collections import defaultdict

import pandas as pd
from rdflib import Graph, URIRef
from rdflib.namespace import RDF, RDFS

sys.path.insert(0, str(Path.cwd()))
from graph_loader import load_graph_from_ttl, DEFAULT_GRAPH_DIR, SG

g = load_graph_from_ttl(DEFAULT_GRAPH_DIR)
print(f"Загружено триплов: {len(g)}")

Загружено триплов: 480055


## 2. Вспомогательные функции

Определяем тезаурус по URI и получаем подпись для отображения.

In [2]:
def thesaurus_name(uri):
    if uri is None:
        return None
    s = str(uri)
    if "fme" in s:
        return "FME"
    if "me" in s:
        return "ME"
    return s

def get_label(g, subject, default=None):
    o = g.value(subject, RDFS.label, None)
    if o is not None:
        return str(o)
    return default or str(subject)

---
## 3. Авторы (Person)

- **Статьи:** Person → `sg:hasArticle` → Article  
- **Публикации:** Person → `sg:hasPublication` → Publication  
- **Тезаурус** автора = тезаурусы всех его статей (`Article hasThesaurus`).

In [3]:
# Количество статей и публикаций на автора
person_articles = defaultdict(int)
person_publications = defaultdict(int)
person_thesauri = defaultdict(set)  # множество тезаурусов (FME/ME) по статьям автора

for person, _, article in g.triples((None, SG.hasArticle, None)):
    if (person, RDF.type, SG.Person) in g and (article, RDF.type, SG.Article) in g:
        person_articles[str(person)] += 1
        th = g.value(article, SG.hasThesaurus, None)
        if th is not None:
            person_thesauri[str(person)].add(thesaurus_name(th))

for person, _, pub in g.triples((None, SG.hasPublication, None)):
    if (person, RDF.type, SG.Person) in g:
        person_publications[str(person)] += 1

person_labels = {p: get_label(g, URIRef(p)) for p in set(person_articles) | set(person_publications)}
print(f"Авторов (с хотя бы одной статьёй или публикацией): {len(person_labels)}")

Авторов (с хотя бы одной статьёй или публикацией): 4851


### Топ-10 авторов по статьям

In [4]:
top_author_articles = sorted(person_articles.items(), key=lambda x: -x[1])[:10]
display(pd.DataFrame([
    {"Автор": person_labels.get(p, p), "URI": p, "Число статей": n}
    for p, n in top_author_articles
]))

,Автор,URI,Число статей
0,М. И. Войцеховский,https://scilib.ai/id/person-5405df60-3ad0-5685...,97
1,А. Б. Иванов,https://scilib.ai/id/person-915053d9-c8ee-51f9...,86
2,Е. Д. Соломенцев,https://scilib.ai/id/person-c992b7b5-ae44-5d36...,61
3,Ю. А. Неретин,https://scilib.ai/id/person-f45a89b9-a86b-5b51...,48
4,Д. В. Аносов,https://scilib.ai/id/person-fc390b4d-49c3-5514...,47
5,Д. Д. Соколов,https://scilib.ai/id/person-225b7cac-eb8d-5c8e...,47
6,М. Э. Казарян,https://scilib.ai/id/person-4d839a59-869d-5ba6...,38
7,Д. П. Санкович,https://scilib.ai/id/person-69025fa7-1a12-5408...,29
8,М. И. Войчеховский,https://scilib.ai/id/person-6e546d21-c7cc-530f...,27
9,О. А. Иванова,https://scilib.ai/id/person-81267ae5-a946-558b...,25


### Топ-10 авторов по публикациям

In [5]:
top_author_pubs = sorted(person_publications.items(), key=lambda x: -x[1])[:10]
display(pd.DataFrame([
    {"Автор": person_labels.get(p, p), "URI": p, "Число публикаций": n}
    for p, n in top_author_pubs
]))

,Автор,URI,Число публикаций
0,Бурбаки Н.,https://scilib.ai/id/person-e16c2e8c-082c-59df...,54
1,Боголюбов Н. Н.,https://scilib.ai/id/person-0543fe39-a42b-5c08...,47
2,Ландау Л. Д.,https://scilib.ai/id/person-7eb799e8-611e-5604...,40
3,Арнольд В. И.,https://scilib.ai/id/person-13c5b332-838d-5537...,35
4,Лифшиц Е. М.,https://scilib.ai/id/person-f67f06af-21b7-5916...,32
5,Владимиров В. С.,https://scilib.ai/id/person-1af719ce-b5cb-5e61...,26
6,АЛександров П. С.,https://scilib.ai/id/person-dc0b7b92-528d-582d...,26
7,Данфорд Н.,https://scilib.ai/id/person-a1211c78-2f38-5501...,24
8,Феллер В.,https://scilib.ai/id/person-782d4b77-f028-5a98...,23
9,Гильберт Д.,https://scilib.ai/id/person-02a83d6e-84b4-54ae...,21


### Среднее количество статей и публикаций на автора

In [6]:
n_authors = len(person_labels)
avg_articles = sum(person_articles.values()) / n_authors if n_authors else 0
avg_publications = sum(person_publications.values()) / n_authors if n_authors else 0
print(f"Среднее число статей на автора:    {avg_articles:.2f}")
print(f"Среднее число публикаций на автора: {avg_publications:.2f}")

Среднее число статей на автора:    0.72
Среднее число публикаций на автора: 1.26


### Авторы: только в одном тезаурусе vs в нескольких

In [7]:
authors_one_th = sum(1 for p in person_labels if len(person_thesauri.get(p, set())) == 1)
authors_multi_th = sum(1 for p in person_labels if len(person_thesauri.get(p, set())) > 1)
authors_no_th = sum(1 for p in person_labels if len(person_thesauri.get(p, set())) == 0)
print("Распределение авторов по охвату тезаурусов:")
print(f"  Только в одном тезаурусе:  {authors_one_th}")
print(f"  В нескольких тезаурусах:  {authors_multi_th}")
print(f"  Без статей с тезаурусом:  {authors_no_th}")

Распределение авторов по охвату тезаурусов:
  Только в одном тезаурусе:  1309
  В нескольких тезаурусах:  56
  Без статей с тезаурусом:  3486


---
## 4. Публикации (Publication)

- **Использование в статьях:** Article → `sg:refersTo` → Publication (сколько статей ссылаются на публикацию).  
- **Тезаурус** публикации = тезаурусы статей, которые на неё ссылаются.

In [8]:
pub_usage = defaultdict(int)   # сколько статей ссылаются на публикацию
pub_thesauri = defaultdict(set)

for article, _, pub in g.triples((None, SG.refersTo, None)):
    if (pub, RDF.type, SG.Publication) in g:
        pub_usage[str(pub)] += 1
        th = g.value(article, SG.hasThesaurus, None)
        if th is not None:
            pub_thesauri[str(pub)].add(thesaurus_name(th))

pub_labels = {p: get_label(g, URIRef(p)) for p in pub_usage}
print(f"Публикаций (использованных хотя бы в одной статье): {len(pub_labels)}")

Публикаций (использованных хотя бы в одной статье): 5689


### Топ-10 публикаций по использованию в статьях

In [9]:
top_pub_usage = sorted(pub_usage.items(), key=lambda x: -x[1])[:10]
display(pd.DataFrame([
    {"Публикация": (pub_labels.get(p, p) or p)[:80], "URI": p, "Число использований": n}
    for p, n in top_pub_usage
]))

,Публикация,URI,Число использований
0,Плоские кривые,https://scilib.ai/id/publication-6fa8ae05-d9dc...,32
1,Введение в теорию квантованных полей,https://scilib.ai/id/publication-8f6893ef-292c...,29
2,Геометрическая теория функций комплексного пер...,https://scilib.ai/id/publication-5c964508-c35e...,23
3,Статистическая механика. Строгие результаты,https://scilib.ai/id/publication-d2607287-e142...,18
4,Методы теории функций многих комплексных перем...,https://scilib.ai/id/publication-0c1b07f6-f652...,17
5,Введение в метаматематику,https://scilib.ai/id/publication-b3eba311-4bf3...,16
6,Введение в теорию множеств и общую топологию,https://scilib.ai/id/publication-3e3a2b8c-ebe6...,15
7,Математические методы классической механики,https://scilib.ai/id/publication-262ed17c-3ff2...,13
8,Равновесная и неравновесная статистическая мех...,https://scilib.ai/id/publication-d6ac9246-d14b...,13
9,Алгебра,https://scilib.ai/id/publication-c1437278-be50...,13


### Среднее количество использований публикации

In [10]:
n_pubs = len(pub_labels)
avg_pub_usage = sum(pub_usage.values()) / n_pubs if n_pubs else 0
print(f"Среднее число использований на публикацию: {avg_pub_usage:.2f}")

Среднее число использований на публикацию: 1.26


### Публикации: только в одном тезаурусе vs в нескольких

In [11]:
pubs_one_th = sum(1 for p in pub_labels if len(pub_thesauri.get(p, set())) == 1)
pubs_multi_th = sum(1 for p in pub_labels if len(pub_thesauri.get(p, set())) > 1)
pubs_no_th = sum(1 for p in pub_labels if len(pub_thesauri.get(p, set())) == 0)
print("Распределение публикаций по охвату тезаурусов:")
print(f"  Только в одном тезаурусе:  {pubs_one_th}")
print(f"  В нескольких тезаурусах:  {pubs_multi_th}")
print(f"  Без привязки к тезаурусу: {pubs_no_th}")

Распределение публикаций по охвату тезаурусов:
  Только в одном тезаурусе:  5578
  В нескольких тезаурусах:  111
  Без привязки к тезаурусу: 0


---
## 5. Формулы (Formula)

- **Использование в статьях:** Article → `sg:hasFormula` → Formula (сколько статей используют формулу).  
- **Тезаурус** формулы = тезаурусы статей, в которых она встречается.

In [12]:
formula_usage = defaultdict(int)
formula_thesauri = defaultdict(set)

for article, _, formula in g.triples((None, SG.hasFormula, None)):
    if (formula, RDF.type, SG.Formula) in g:
        formula_usage[str(formula)] += 1
        th = g.value(article, SG.hasThesaurus, None)
        if th is not None:
            formula_thesauri[str(formula)].add(thesaurus_name(th))

formula_labels = {f: get_label(g, URIRef(f)) for f in formula_usage}
print(f"Формул (использованных хотя бы в одной статье): {len(formula_labels)}")

Формул (использованных хотя бы в одной статье): 24902


### Топ-10 формул по использованию в статьях

In [13]:
top_formula_usage = sorted(formula_usage.items(), key=lambda x: -x[1])[:10]
display(pd.DataFrame([
    {"Формула (label/URI)": (formula_labels.get(f, f) or f)[:60], "URI": f, "Число использований": n}
    for f, n in top_formula_usage
]))

,Формула (label/URI),URI,Число использований
0,\mu(A)=\mu\left(T^{-1} A\right),http://libmeta.ru/fme/formula/main/1000_1_INVA...,1
1,\int f(x) d \mu(x)=\int f(T x) d \mu(x) .,http://libmeta.ru/fme/formula/main/1000_2_INVA...,1
2,\int_{X} f \cdot \omega=\int_{X} T_{g} f \cdot...,http://libmeta.ru/fme/formula/main/1002_1_INVA...,1
3,T_{g} f(x)=f\left(g^{-1} x\right),http://libmeta.ru/fme/formula/main/1002_2_INVA...,1
4,\int_{G} f(g) d \mu(g) .,http://libmeta.ru/fme/formula/main/1002_3_INVA...,1
5,\begin{aligned}d \mu_{l}(g h) & =\Delta_{G}(h)...,http://libmeta.ru/fme/formula/main/1002_4_INVA...,1
6,"\theta_{i}=\sum_{j} \theta_{i j}(x) d x_{j}, \...",http://libmeta.ru/fme/formula/main/1002_5_INVA...,1
7,\begin{gathered}\bar{g}=g \Gamma\left(\left\{p...,http://libmeta.ru/fme/formula/main/1009_1_INVA...,1
8,"(\boldsymbol{x}, \boldsymbol{y})=\sum_{i, j} g...",http://libmeta.ru/fme/formula/main/1014_1_INDE...,1
9,"(\boldsymbol{x}, \boldsymbol{y})=\sum_{i, j} g...",http://libmeta.ru/fme/formula/main/1014_2_INDE...,1


### Среднее количество использований формулы

In [14]:
n_formulas = len(formula_labels)
avg_formula_usage = sum(formula_usage.values()) / n_formulas if n_formulas else 0
print(f"Среднее число использований на формулу: {avg_formula_usage:.2f}")

Среднее число использований на формулу: 1.00


### Формулы: только в одном тезаурусе vs в нескольких

In [15]:
formulas_one_th = sum(1 for f in formula_labels if len(formula_thesauri.get(f, set())) == 1)
formulas_multi_th = sum(1 for f in formula_labels if len(formula_thesauri.get(f, set())) > 1)
formulas_no_th = sum(1 for f in formula_labels if len(formula_thesauri.get(f, set())) == 0)
print("Распределение формул по охвату тезаурусов:")
print(f"  Только в одном тезаурусе:  {formulas_one_th}")
print(f"  В нескольких тезаурусах:  {formulas_multi_th}")
print(f"  Без привязки к тезаурусу: {formulas_no_th}")

Распределение формул по охвату тезаурусов:
  Только в одном тезаурусе:  24902
  В нескольких тезаурусах:  0
  Без привязки к тезаурусу: 0
